In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<!-- # Paramètres de la pipeline -->

In [ ]:
code_departement = "093"

<!-- # Imports & setup -->

In [70]:
import geopandas as gpd

from potentiel_solaire.attach_buildings_to_schools import attach_buildings_to_schools
from potentiel_solaire.constants import DEFAULT_CRS, RESULTS_FOLDER
from potentiel_solaire.sources.bd_topo import extract_bd_topo, get_topo_zones_of_interest, \
    get_topo_buildings_of_interest
from potentiel_solaire.sources.extract import extract_sources
from potentiel_solaire.sources.schools_establishments import get_schools_establishments_of_interest
from potentiel_solaire.sources.protected_buildings import get_areas_with_protected_buildings
from potentiel_solaire.features.solar_potential import calculate_solar_potential
from potentiel_solaire.aggregate import aggregate_solar_potential_by_etablishment
from potentiel_solaire.duckdb_manager import save_solar_potential_by_school, \
    save_solar_potential_by_commune, \
    save_solar_potential_by_department
from potentiel_solaire.geojson_files_creator import export_to_geofile

<!-- # Extraction des données sources -->

<!-- ### Telechargements des fichiers listes dans data/sources.yaml -->

In [71]:
sources = extract_sources()
for name, source in sources.items():
    print(f"source {name} extracted at {source.filepath}")

2025-04-03 11:10:50,146 - INFO - root - D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\sources\extract.py - extract_sources - source etablissements already extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\fr-en-annuaire-education.geojson
2025-04-03 11:10:50,148 - INFO - root - D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\sources\extract.py - extract_sources - source immeubles_proteges already extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\liste_immeubles_proteges.geojson
2025-04-03 11:10:50,151 - INFO - root - D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\sources\extract.py - extract_sources - source communes already extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\communes.json
2025-04-03 11:10:50,153 - INFO - root - D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\sources\extract.py - extract_sources - source departements already extracted at D:\datafrogood\13_po

source etablissements extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\fr-en-annuaire-education.geojson
source immeubles_proteges extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\liste_immeubles_proteges.geojson
source communes extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\communes.json
source departements extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\departements.geojson
source regions extracted at D:\datafrogood\13_potentiel_solaire\algorithme\data\regions.json


<!-- ### BD TOPO -->

In [72]:
bd_topo_path = extract_bd_topo(code_departement=code_departement)
print(f"BD TOPO extraite ici: {bd_topo_path}")

2025-04-03 11:10:50,394 - INFO - root - D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\sources\bd_topo.py - extract_bd_topo - gpkg file D:\datafrogood\13_potentiel_solaire\algorithme\data\BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15\BDTOPO\1_DONNEES_LIVRAISON_2024-12-00134\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg for departement 093 already extracted


BD TOPO extraite ici: D:\datafrogood\13_potentiel_solaire\algorithme\data\BDTOPO_3-4_TOUSTHEMES_GPKG_LAMB93_D093_2024-12-15\BDTOPO\1_DONNEES_LIVRAISON_2024-12-00134\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15\BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg


<!-- # Filtre des données sur le périmètre du calcul -->

<!-- ### Etablissements scolaires -->


In [73]:
schools_establishments = get_schools_establishments_of_interest(
    schools_filepath=sources["etablissements"].filepath,
    code_departement=code_departement,
    types_etablissements=['Ecole', 'Lycée', 'Collège'],
    statut_public_prive="Public",
    etat="OUVERT",
    crs=DEFAULT_CRS
)
nb_schools = schools_establishments.shape[0]
print(f"Nb d'établissements scolaires: {nb_schools}")

Nb d'établissements scolaires: 1130


<!-- ### Zone d'intérêt géographique -->

In [74]:
communes = gpd.read_file(bd_topo_path, layer="commune").to_crs(DEFAULT_CRS)
geom_of_interest = communes.sjoin(schools_establishments).dissolve()[["geometry"]]

<!-- ### Zones d'éducations -->

In [75]:
educational_zones = get_topo_zones_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    categories=["Science et enseignement"],
    natures=['Collège', 'Lycée', 'Enseignement primaire'],
    crs=DEFAULT_CRS
)
nb_educational_zones = educational_zones.shape[0]
print("Nb de zones d'éducations: ", nb_educational_zones)

d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_numerotee_ou_nommee', 'itineraire_autre', 'tro

Nb de zones d'éducations:  1088


<!-- ### Bâtiments -->

In [76]:
buildings = get_topo_buildings_of_interest(
    bd_topo_path=bd_topo_path,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)
nb_buildings = buildings.shape[0]
print("Nb de batiments: ", nb_buildings)

d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(256)' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'character varying(30)' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\core.py:279: RuntimeWarning: Field format 'timestamp with time zone' not supported
  return ogr_read_info(
d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\geopandas\io\file.py:497: UserWarning: More than one layer found in 'BDT_3-4_GPKG_LAMB93_D093-ED2024-12-15.gpkg': 'troncon_de_route' (default), 'route_numerotee_ou_nommee', 'itineraire_autre', 'tro

Nb de batiments:  351576


<!-- ### Zones avec des bâtiments protégés -->

In [77]:
areas_with_protected_buildings = get_areas_with_protected_buildings(
    bd_protected_buildings_path=sources["immeubles_proteges"].filepath,
    geom_of_interest=geom_of_interest,
    crs=DEFAULT_CRS
)

<!-- # Détermination des bâtiments scolaires -->

In [78]:
schools_buildings = attach_buildings_to_schools(
    schools_establishments=schools_establishments,
    educational_zones=educational_zones,
    buildings=buildings
)
nb_schools_buildings = schools_buildings.shape[0]
print("Nb de batiments scolaires: ", nb_schools_buildings)

D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\attach_buildings_to_schools.py:23: UserWarning: `keep_geom_type=True` in overlay resulted in 162 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  intersections = gpd.overlay(
D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\attach_buildings_to_schools.py:43: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  educational_zones_merged["cleabs_grande_zone"].fillna(educational_zones_merged["

Nb de batiments scolaires:  4261


<!-- # Calcul des attributs utiles pour le potentiel solaire -->

In [79]:
solar_potential_of_schools_buildings = calculate_solar_potential(
    schools_buildings=schools_buildings,
    areas_with_protected_buildings=areas_with_protected_buildings,
    geom_of_interest=geom_of_interest
)


D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\features\solar_potential.py:51: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = Point(geom_of_interest.centroid.x, geom_of_interest.centroid.y)
D:\datafrogood\13_potentiel_solaire\algorithme\potentiel_solaire\features\solar_potential.py:52: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schools_buildings["distance_to_center"] = schools_buildings.geometry.distance(center)


<!-- # Evaluation du Potentiel solaire par Etablissement -->

In [82]:
results_by_school = aggregate_solar_potential_by_etablishment(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings
)

results_by_school.head()

,identifiant_de_l_etablissement,surface_utile,potentiel_solaire,protection
0,0930043S,2261.489038,5.548897e+05,False
1,0930089S,3245.297871,7.962817e+05,True
2,0930100D,2069.812266,5.078590e+05,False
3,0930116W,2563.574939,6.290109e+05,True
4,0930117X,4907.794034,1.204200e+06,True


<!-- # Checks sur la qualité des données & calculs -->

In [83]:
nb_schools_with_buildings = len(schools_buildings.identifiant_de_l_etablissement.unique())
print("Nb d'établissements scolaires avec des batiments: {} ({}%)".format(
    nb_schools_with_buildings,
    round(100 * nb_schools_with_buildings / nb_schools)
))

Nb d'établissements scolaires avec des batiments: 694 (61%)


In [84]:
nb_buildings_protected = solar_potential_of_schools_buildings[solar_potential_of_schools_buildings.protection].shape[0]
print(f"Nb de batiments protégés {nb_buildings_protected} ({100*nb_buildings_protected/nb_schools_buildings:.2f}%)")

Nb de batiments protégés 1134 (26.61%)


<!-- ### Evaluation du Potentiel solaire par Etablissement -->

In [85]:
results_by_school = aggregate_solar_potential_by_etablishment(
    schools_establishments=schools_establishments,
    solar_potential_of_schools_buildings=solar_potential_of_schools_buildings
)


<!-- # Sauvegarde des calculs en Base de Données -->

<!-- #### Mise à jour des données des Etablissements du Département en Base de Données -->

In [86]:
stats_schools= save_solar_potential_by_school(
    results_by_school,
    code_departement
)

assert stats_schools.query('surface_utile > 0').shape[0] > 0, "Problème lors de la mise à jour des établissements"

stats_schools.head(3)

,identifiant_de_l_etablissement,nom_etablissement,type_etablissement,libelle_nature,code_commune,nom_commune,code_departement,libelle_departement,code_region,libelle_region,surface_utile,potentiel_solaire,protection,geo_point,geometry
0,0930171F,Ecole élémentaire Diderot 1,Ecole,ECOLE DE NIVEAU ELEMENTAIRE,93029,Drancy,093,Seine-Saint-Denis,11,Ile-de-France,1991,488452,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{""type"":""Point"",""coordinates"":[2.4260302949154..."
1,0930173H,Ecole élémentaire Marcel Cachin,Ecole,ECOLE DE NIVEAU ELEMENTAIRE,93055,Pantin,093,Seine-Saint-Denis,11,Ile-de-France,1304,320031,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{""type"":""Point"",""coordinates"":[2.4136966799138..."
2,0930194F,Ecole élémentaire Jean Macé,Ecole,ECOLE DE NIVEAU ELEMENTAIRE,93001,Aubervilliers,093,Seine-Saint-Denis,11,Ile-de-France,0,0,False,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","{""type"":""Point"",""coordinates"":[2.3897234458295..."


<!-- #### Mise à jour des données des Communes du Département en Base de Données -->

In [87]:
stats_communes = save_solar_potential_by_commune(
    code_departement
)

assert stats_communes.query('surface_utile > 0').shape[0] > 0, "Problème lors de la mise à jour des communes"

stats_communes.head(3)

,code_commune,nom_commune,code_departement,libelle_departement,code_region,libelle_region,surface_utile,potentiel_solaire,count_etablissements,count_etablissements_proteges,geom,geometry
0,93001,Aubervilliers,093,Seine-Saint-Denis,11,Île-de-France,52912,12982361,49,12,"[2, 4, 0, 0, 0, 0, 0, 0, 170, 102, 23, 64, 209...","{""type"":""Polygon"",""coordinates"":[[[2.389324795..."
1,93005,Aulnay-sous-Bois,093,Seine-Saint-Denis,11,Île-de-France,71801,17617972,65,3,"[2, 4, 0, 0, 0, 0, 0, 0, 224, 104, 29, 64, 232...","{""type"":""Polygon"",""coordinates"":[[[2.507176188..."
2,93006,Bagnolet,093,Seine-Saint-Denis,11,Île-de-France,22736,5578014,24,7,"[2, 4, 0, 0, 0, 0, 0, 0, 233, 116, 26, 64, 187...","{""type"":""Polygon"",""coordinates"":[[[2.415272860..."


<!-- #### Mise à jour des données du Département en Base de données -->

In [88]:
stats_department = save_solar_potential_by_department(
    code_departement
)

assert stats_department['surface_utile'].shape[0] > 0, f"Problème lors de la mise à jour du département {code_departement}"

stats_department.head(3)

,code_departement,libelle_departement,code_region,libelle_region,surface_utile,potentiel_solaire,count_etablissements,count_etablissements_proteges,geom,geometry
0,093,Seine-Saint-Denis,11,Île-de-France,1153319,282983251,1130,193,"[2, 4, 0, 0, 0, 0, 0, 0, 254, 115, 18, 64, 159...","{""type"":""Polygon"",""coordinates"":[[[2.319887543..."


<!-- # Dump des donnees pour analyses -->

In [89]:
layers = ["schools_establishments", "educational_zones", "schools_buildings", "solar_potential_of_schools_buildings", "areas_with_protected_buildings"]
gdfs = [schools_establishments, educational_zones, schools_buildings, solar_potential_of_schools_buildings, areas_with_protected_buildings]
output_gpkg = RESULTS_FOLDER / f"D{code_departement}_pipeline_results.gpkg"

for layer, gdf in zip(layers, gdfs):
    gdf.to_file(output_gpkg, layer=layer, driver="GPKG")

export_to_geofile(stats_schools, output_gpkg, layer="etablissements", driver="GPKG")
export_to_geofile(stats_communes, output_gpkg, layer="communes", driver="GPKG")
export_to_geofile(stats_department, output_gpkg, layer="departements", driver="GPKG")

2025-04-03 11:14:06,862 - INFO - pyogrio._io - d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - write - Created 1,130 records
2025-04-03 11:14:06,981 - INFO - pyogrio._io - d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - write - Created 1,088 records
2025-04-03 11:14:07,257 - INFO - pyogrio._io - d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - write - Created 4,261 records
2025-04-03 11:14:07,565 - INFO - pyogrio._io - d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - write - Created 4,261 records
2025-04-03 11:14:07,680 - INFO - pyogrio._io - d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - write - Created 88 records
2025-04-03 11:14:07,850 - INFO - pyogrio._io - d:\datafrogood\13_potentiel_solaire\algorithme\.venv\Lib\site-packages\pyogrio\raw.py - write - Created 1,130 records
2025-04-03 11

True